In [2]:
import pandas as pd
import numpy as np

In [3]:
column_name = ['user_id' , 'item_id' , 'rating' , 'timestamp']
rating = pd.read_csv('dataset/u.data' , sep='\t', names= column_name)
rating.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [4]:
rating.shape

(100000, 4)

In [5]:
print(rating['user_id'].nunique(),'unique users')
print(rating['item_id'].nunique(),'unique movies')

943 unique users
1682 unique movies


User Item rating matrix

In [6]:
user_rating_matrix = rating.pivot(index='user_id' , columns='item_id' , values='rating')
user_rating_matrix = user_rating_matrix.fillna(user_rating_matrix.mean(axis=1))
user_rating_matrix_filled = user_rating_matrix.fillna(0)
print(user_rating_matrix)

item_id      1         2         3         4         5         6         7     \
user_id                                                                         
1        5.000000  3.000000  4.000000  3.000000  3.000000  5.000000  4.000000   
2        4.000000  3.709677  2.796296  4.333333  2.874286  3.635071  3.965261   
3        3.610294  3.709677  2.796296  4.333333  2.874286  3.635071  3.965261   
4        3.610294  3.709677  2.796296  4.333333  2.874286  3.635071  3.965261   
5        4.000000  3.000000  2.796296  4.333333  2.874286  3.635071  3.965261   
...           ...       ...       ...       ...       ...       ...       ...   
939      3.610294  3.709677  2.796296  4.333333  2.874286  3.635071  3.965261   
940      3.610294  3.709677  2.796296  2.000000  2.874286  3.635071  4.000000   
941      5.000000  3.709677  2.796296  4.333333  2.874286  3.635071  4.000000   
942      3.610294  3.709677  2.796296  4.333333  2.874286  3.635071  3.965261   
943      3.610294  5.000000 

Now we ar going to find the similarity between users we are goint to use cosine similarity 

In [7]:
from sklearn.metrics.pairwise import cosine_similarity
user_similarity = cosine_similarity(user_rating_matrix_filled)
print(user_similarity)

[[1.         0.97823677 0.97657416 ... 0.97951346 0.97371924 0.96886315]
 [0.97823677 1.         0.99148667 ... 0.99406743 0.98969777 0.98287126]
 [0.97657416 0.99148667 1.         ... 0.99262222 0.98730806 0.98051563]
 ...
 [0.97951346 0.99406743 0.99262222 ... 1.         0.99024605 0.98386724]
 [0.97371924 0.98969777 0.98730806 ... 0.99024605 1.         0.97948439]
 [0.96886315 0.98287126 0.98051563 ... 0.98386724 0.97948439 1.        ]]


In [8]:
user_similarity_df = pd.DataFrame(user_similarity, index=user_rating_matrix.index, columns=user_rating_matrix.index)
print(user_similarity_df.head())

user_id       1         2         3         4         5         6         7    \
user_id                                                                         
1        1.000000  0.978237  0.976574  0.979689  0.967695  0.974432  0.967206   
2        0.978237  1.000000  0.991487  0.995353  0.977879  0.987656  0.977213   
3        0.976574  0.991487  1.000000  0.992742  0.975444  0.984403  0.975528   
4        0.979689  0.995353  0.992742  1.000000  0.979777  0.987801  0.977531   
5        0.967695  0.977879  0.975444  0.979777  1.000000  0.972307  0.964784   

user_id       8         9         10   ...       934       935       936  \
user_id                                ...                                 
1        0.980319  0.979747  0.977792  ...  0.969793  0.978171  0.958259   
2        0.991799  0.995603  0.991786  ...  0.983373  0.993739  0.971289   
3        0.990153  0.993326  0.988668  ...  0.980905  0.991788  0.968571   
4        0.994032  0.997789  0.992481  ...  0.984792

In [9]:
target_user = 10

In [10]:
similar_user = user_similarity_df[target_user].sort_values(ascending=False)
print(similar_user)

user_id
10     1.000000
522    0.994049
874    0.993971
156    0.993674
573    0.993667
         ...   
181    0.939745
13     0.936607
279    0.931933
655    0.900470
405    0.878569
Name: 10, Length: 943, dtype: float64


Most Similar users

In [11]:
most_similar_user_to_target_user = similar_user.index[1:11]
print(most_similar_user_to_target_user)

Index([522, 874, 156, 573, 19, 138, 322, 876, 33, 607], dtype='int64', name='user_id')


How many movies target user rated

In [12]:
target_user_rated_movies = user_rating_matrix.loc[target_user].dropna().index
print(target_user_rated_movies)

Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       934, 935, 936, 937, 938, 939, 940, 941, 942, 943],
      dtype='int64', name='item_id', length=943)


Get movies from similar users

In [13]:
Similar_user_rating = user_rating_matrix.loc[most_similar_user_to_target_user]
print(Similar_user_rating)

item_id      1         2         3         4         5         6         7     \
user_id                                                                         
522      3.610294  3.709677  2.796296  4.333333  2.874286  3.635071  3.965261   
874      3.610294  3.709677  2.796296  4.333333  2.874286  3.635071  3.965261   
156      3.610294  3.709677  2.796296  4.333333  2.874286  3.635071  3.965261   
573      3.610294  3.709677  2.796296  4.333333  2.874286  3.635071  3.965261   
19       3.610294  3.709677  2.796296  4.000000  2.874286  3.635071  3.965261   
138      4.000000  3.709677  2.796296  4.333333  2.874286  3.635071  3.965261   
322      2.000000  3.709677  2.796296  4.333333  2.874286  3.635071  3.965261   
876      3.610294  3.709677  2.796296  4.333333  2.874286  3.635071  3.965261   
33       3.610294  3.709677  2.796296  4.333333  2.874286  3.635071  3.965261   
607      3.610294  3.709677  2.796296  4.333333  2.874286  3.635071  3.965261   

item_id     8         9    

In [14]:
unrated_movies = user_rating_matrix.loc[target_user][user_rating_matrix.loc[target_user].isna()].index
print(unrated_movies)

Index([ 944,  945,  946,  947,  948,  949,  950,  951,  952,  953,
       ...
       1673, 1674, 1675, 1676, 1677, 1678, 1679, 1680, 1681, 1682],
      dtype='int64', name='item_id', length=739)


In [15]:
similarities = similar_user[most_similar_user_to_target_user]
print(similarities)

user_id
522    0.994049
874    0.993971
156    0.993674
573    0.993667
19     0.993637
138    0.993612
322    0.993517
876    0.993474
33     0.993471
607    0.993399
Name: 10, dtype: float64


In [16]:
predicted_ratings = {}

for movie in unrated_movies:
    movie_rating = user_rating_matrix.loc[most_similar_user_to_target_user , movie]
    similarities = similar_user[most_similar_user_to_target_user]

    numerator = (similarities * movie_rating).sum()
    denominator = similarities[movie_rating.notna()].sum()
    predicted_value = numerator/denominator if denominator != 0 else 0

    predicted_ratings[movie] = predicted_value


In [19]:
recommendations = sorted(predicted_ratings.items(), key=lambda x: x[1], reverse=True)[:5]
print(recommendations)


[(1019, np.float64(4.0)), (950, np.float64(3.0)), (1012, np.float64(2.0)), (944, 0), (945, 0)]
